# Degridder experiment

Here, we're comparing with the Preesm implementation available at https://gitlab.insa-rennes.fr/Anaelle.Cloarec/degridder

First, let's run and measure all scenarios.

In [ ]:
experiment_dir = !realpath ~/repos/iara/experiment/degridder
experiment_dir = experiment_dir[0]

# take only PXX_ scenarios, sort by number

all_scenarios = !ls ~/repos/degridder/Scenarios
all_scenarios = list(filter(lambda x: str(x).startswith("P"), all_scenarios))
all_scenarios.sort(key=lambda x: int(str(x).split('_')[0].strip('P')))

In [9]:


%cd {experiment_dir}
!rm -rf instances
!mkdir -p instances


for scenario in all_scenarios:
  !mkdir -p "{experiment_dir}/instances/{scenario}"
  !mkdir -p "{experiment_dir}/instances/{scenario}/build"
  !rm -rf "{experiment_dir}/instances/{scenario}/build/*"
  ! \time -v -o "{experiment_dir}/instances/{scenario}/time.txt" timeout 5m ~/repos/preesm-cli/commandLinePreesm.sh ~/Downloads/preesm-3.21.0.202501251928-linux.gtk.x86_64/ ~/repos/degridder/ Codegen.workflow {scenario}
  %cd ~/repos/degridder/Code
  ! cmake -B build
  %cd build
  ! make
  !ls
  !pwd
  ! cp degridder_pipeline  {experiment_dir}/instances/{scenario}/build/degridder_pipeline_{scenario.removesuffix('.scenario')}


/home/jabcross/repos/iara/experiment/degridder

***START*** Sun, 13 Jul 2025 18:45:13 -0300


Init workspace and import project

CompileCommand: exclude org/eclipse/jdt/internal/core/dom/rewrite/ASTRewriteAnalyzer.getExtendedRange bool exclude = true
Create.
Opening 'degridder'.

Run workflow from project degridder

CompileCommand: exclude org/eclipse/jdt/internal/core/dom/rewrite/ASTRewriteAnalyzer.getExtendedRange bool exclude = true
18:45:21 NOTICE: Starting workflow execution: degridder/Workflows/Codegen.workflow.18:45:22 NOTICE: Workflow Step: 'PiMM2SrDAG' (id 'pisdf-srdag'): [PiSDFToSingleRateTask] Transforming PiGraph to Single-Rate Directed Acyclic PiGraph.18:45:22 NOTICE: Computing Repetition Vector for graph [top_degridder]18:45:22 NOTICE: top_degridder/config_struct_set_up x1 (total: x1)
18:45:22 NOTICE: top_degridder/broadcast_config x1 (total: x1)
18:45:22 NOTICE: top_degridder/degridder_parallel x1 (total: x1)
18:45:22 NOTICE: top_degridder/fft_shift_complex_to_complex_ac

In [10]:
# get binary sizes

import os

binaries = !ls {experiment_dir}/instances/*/build/*

list(zip(binaries,map(os.path.getsize, binaries)))

[('/home/jabcross/repos/iara/experiment/degridder/instances/P11_large_1_8_1_complete.scenario/build/degridder_pipeline_P11_large_1_8_1_complete',
  1122712),
 ('/home/jabcross/repos/iara/experiment/degridder/instances/P11_large_1_8_1.scenario/build/degridder_pipeline_P11_large_1_8_1',
  1115544),
 ('/home/jabcross/repos/iara/experiment/degridder/instances/P12_large_32_8_64.scenario/build/degridder_pipeline_P12_large_32_8_64',
  1456336),
 ('/home/jabcross/repos/iara/experiment/degridder/instances/P13_large_32_8_128.scenario/build/degridder_pipeline_P13_large_32_8_128',
  1696848),
 ('/home/jabcross/repos/iara/experiment/degridder/instances/P14_large_64_8_128.scenario/build/degridder_pipeline_P14_large_64_8_128',
  1803272),
 ('/home/jabcross/repos/iara/experiment/degridder/instances/P15_large_64_8_256.scenario/build/degridder_pipeline_P15_large_64_8_256',
  1803272),
 ('/home/jabcross/repos/iara/experiment/degridder/instances/P1_bis_medium_1_8_1.scenario/build/degridder_pipeline_P1_bis

Now, let's run our version.

In [43]:
all_scenarios

parameters = {}

# These refer to the input file, not the Preesm scenarios.
scenario_numbers = {
  "small": 1,
  "medium": 2,
  "large": 3
}

%cd {experiment_dir}

for scenario in all_scenarios:
  size, cores, num_kernel_support, num_chunks = scenario.removesuffix(".scenario").removesuffix("_complete").split("_")[-4:]
  print(size,
  cores,
  num_kernel_support,
  num_chunks)
  GRID_SIZE = 5120 if size == "large" else 2560
  NUM_KERNEL_SUPPORT = int(num_kernel_support)
  OVERSAMPLING_FACTOR = 16
  NUMBER_SAMPLE_IN_KERNEL = ((NUM_KERNEL_SUPPORT + 1) * OVERSAMPLING_FACTOR) ** 2
  NUM_KERNELS = 17
  NUM_SCENARIO = scenario_numbers[size]
  NUM_VISIBILITIES = 7848960 if size == "large" else 3924480
  TOTAL_KERNELS_SAMPLES = NUM_KERNELS * NUMBER_SAMPLE_IN_KERNEL
  NUM_CHUNK = int(num_chunks)
  NUM_VISIB_D_N_CHUNK = NUM_VISIBILITIES // NUM_CHUNK
  assert(NUM_VISIB_D_N_CHUNK * NUM_CHUNK == NUM_VISIBILITIES)
  ! rm -f {experiment_dir}/instances/{scenario}/build/topology.mlir
  ! cat topology.mlir.template \
    | sed 's/GRID_SIZE/{GRID_SIZE}/g' \
    | sed 's/NUM_KERNEL_SUPPORT/{NUM_KERNEL_SUPPORT}/g' \
    | sed 's/OVERSAMPLING_FACTOR/{OVERSAMPLING_FACTOR}/g' \
    | sed 's/NUMBER_SAMPLE_IN_KERNEL/{NUMBER_SAMPLE_IN_KERNEL}/g' \
    | sed 's/NUM_KERNELS/{NUM_KERNELS}/g' \
    | sed 's/NUM_SCENARIO/{NUM_SCENARIO}/g' \
    | sed 's/NUM_VISIBILITIES/{NUM_VISIBILITIES}/g' \
    | sed 's/TOTAL_KERNELS_SAMPLES/{TOTAL_KERNELS_SAMPLES}/g' \
    | sed 's/NUM_CHUNK/{NUM_CHUNK}/g' \
    | sed 's/NUM_VISIB_D_N_CHUNK/{NUM_VISIB_D_N_CHUNK}/g' \
    > {experiment_dir}/instances/{scenario}/build/topology.mlir


/home/jabcross/repos/iara/experiment/degridder
medium 1 8 1
small 1 8 1
small 1 8 1
small 32 8 64
small 32 8 128
small 64 8 128
small 64 8 256
large 1 4 1
large 1 4 1
large 32 4 64
large 32 4 128
large 64 4 128
large 64 4 256
large 1 8 1
large 1 8 1
large 32 8 64
large 32 8 128
large 64 8 128
large 64 8 256
large 64 8 1024
large 64 4 512
small 64 8 512
